In [1]:
from misc.misc import run_in_thread_simple
%load_ext autoreload
%autoreload 2
# %matplotlib notebook
import matplotlib
matplotlib.use('TKAgg')
import os 
import datetime
import numpy as np
import matplotlib.pyplot as plt
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_modes2 import gen_input_spots_array, gen_output_modes_Unitary
from pianoq.misc.misc import run_in_thread, run_in_thread_simple

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_05_QKD_random_phases'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path = fr'{dir_path}\{timestamp}_QKD_MUB2.masks'

## Checklist
- [X] make list
- [X] create masks WFM
- [ ] measure single counts 
- [ ] mark spots locations 
- [ ] measure coincidence no phases
- [ ] find phases
- [ ] measure with good phases 
- [ ] measure with random phases X 5 

# Create masks WFM

In [ ]:
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_QKD_masks
mplc = create_WFM_QKD_masks(N_iterations=5, out_path=masks_path)

In [ ]:
mplc.res._calc_normalized_overlap()
display(np.angle(np.diag(mplc.res.forward_overlap)))
display(np.diag(mplc.res.forward_overlap))
mplc.res._calc_fidelity()
display(mplc.res.fidelity)

In [ ]:
mplc.res.show_overlap()

In [ ]:
fig, axes = plt.subplots(2, 5)
for i, ax in enumerate(axes.flat):
    # ax.imshow(np.angle(res.masks[i][360:720, 140:280]), cmap='gray')
    ax.imshow(np.angle(mplc.res.masks[i][360:720, 140:280]), cmap='gray')

In [ ]:
mplc.res.show_all(7)

# Find locations with single counts 

In [3]:
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan
import numpy as np
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
import glob 

paths = glob.glob(fr'{dir_path}\*.masks')
assert len(paths) == 1, 'there should be one .masks file'
masks_path = paths[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks

mplc = MPLCDevice()
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True)
mplc.restore_location()
# phases_result = PhaseFinderResult()
# phases_result.loadfrom(r"G:\My Drive\People\Ronen\PHD\MPLC\results\2024_08_11_10_26_11_QKD_row3_phases.phases")
# masks = add_phase_input_spots(masks, phases_result.phases)

In [4]:
resolution = 1
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_idler_path = fr'{dir_path}\{timestamp}_QKD_MUB2_signal_res_{resolution}.scan'
singles_signal_path = fr'{dir_path}\{timestamp}_QKD_MUB2_idler_res_{resolution}.scan'

## perform scan

In [6]:
run_in_thread(signal_scan, integration_time=1.0, coin_window=2e-9, resolution=resolution, out_path=singles_signal_path, half_scan=True)

<ThreadWithPrintCapture(Thread-6, started 2196)>

In [7]:
run_in_thread(idler_scan, integration_time=1.0, coin_window=2e-9, resolution=resolution, out_path=singles_idler_path)

<ThreadWithPrintCapture(Thread-8, started 13680)>

In [ ]:
mplc.close()

## mark spots locations  

In [15]:
from pianoq_results.scan_result import ScanResult

path_sig = glob.glob(fr'{dir_path}\*signal_res_1.scan')[0]
path_idl = glob.glob(fr'{dir_path}\*idler_res_1.scan')[0]

res_sig = ScanResult()
res_idl = ScanResult()
res_idl.loadfrom(path_idl)
res_sig.loadfrom(path_sig)
res_idl.show_singles()
res_sig.show_singles()

In [ ]:
res_sig.get_xys(2, saveto_dir=dir_path)
res_idl.get_xys(2, saveto_dir=dir_path)

# Find phases

In [ ]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.find_discreet_phases import PhaseFinder

paths = glob.glob(fr'{dir_path}\*.masks')
assert len(paths) == 1, 'there should be one .masks file'
masks_path = paths[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks

locs_sig_path = glob.glob(fr'{dir_path}\*sig.locs')
locs_idl_path = glob.glob(fr'{dir_path}\*idl.locs')
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']

modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
# masks = add_phase_input_spots(masks, phases_result.phases)
m.load_masks(masks, linear_tilts=True)
initiail_phases = np.zeros(N_SPOTS * 2)

phases_path = fr'{dir_path}\{timestamp}_QKD_MUB2_signal.phases'

# i,j of corr matrix that is supposed to be strong 
i = 0
j = 0
pf = PhaseFinder(mplc=m, integration_time=30, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, intial_phases=initiail_phases, coin_window=2e-9)
pf.m_idl_x.move_absolute(locs_idl[i, 0])
pf.m_idl_y.move_absolute(locs_idl[i, 1])
pf.m_sig_x.move_absolute(locs_sig[j, 0])
pf.m_sig_y.move_absolute(locs_sig[j, 1])
time.sleep(2)

pf.find_phases()

pf.close()
m.close()

# Scan correlations 

In [12]:
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner

masks_path = glob.glob(fr'{dir_path}\*.masks')[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks

modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)

phases_result = PhaseFinderResult()
phases_path = glob.glob(fr'{dir_path}\*.phases')
phases_result.loadfrom(phases_path)

# Important part here! <--------------------------------------
phases_type = 'good'
# phases_type = 'none'
# phases_type = 'random'
if phases_type == 'good':
    phases = add_phase_input_spots(masks, phases_result.phases)
elif phases_type == 'random':
    phases = np.random.uniform(0, 2*np.pi, len(phases_result.phases))
elif phases_type == 'none':
    phases = np.zeros_like(phases_result.phases)
else:
    raise ValueError(f'phases type {phases_type} not recognized')
    
masks = add_phase_input_spots(masks, phases)

locs_sig_path = glob.glob(fr'{dir_path}\*sig.locs')
locs_idl_path = glob.glob(fr'{dir_path}\*idl.locs')
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']

mplc = MPLCDevice()
mplc.load_masks(masks, linear_tilts=True)

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}.scan'
backlash = 0.0
wait_after_move = 0.3
dps = DiscretePhotonScanner(locs_signal, locs_idler, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window)
dps.scan()
dps.close()
mplc.close()


AttributeError: 'list' object has no attribute 'strip'

In [ ]:
dps.res.show()
dps.res.show_singles()